In [5]:
# If using Colab, clone your repo then: %cd personalized-healthcare-assistant
import os, sqlite3, pandas as pd

DB_PATH = os.path.join("..", "data", "EHR.db")
assert os.path.exists(DB_PATH), "EHR.db not found. Run src/sql/load_to_sqlite.py"

In [6]:
conn = sqlite3.connect(DB_PATH)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(tables)

df = pd.read_sql_query("SELECT * FROM asthma_records LIMIT 10;", conn)
display(df.head())

,name
0,asthma_records


,patient_id,age,gender,bmi,smoking_status,family_history,allergies,air_pollution_level,physical_activity_level,occupation_type,comorbidities,medication_adherence,number_of_er_visits,peak_expiratory_flow,feno_level,has_asthma,asthma_control_level
0,ASTH100000,52,Female,27.6,Former,1,None,Moderate,Sedentary,Outdoor,Diabetes,0.38,0,421.0,46.0,0,None
1,ASTH100001,15,Male,24.6,Former,0,Dust,Low,Moderate,Indoor,Both,0.60,2,297.6,22.9,0,None
2,ASTH100002,72,Female,17.6,Never,0,None,Moderate,Moderate,Indoor,None,0.38,0,303.3,15.3,0,None
3,ASTH100003,61,Male,16.8,Never,0,Multiple,High,Sedentary,Outdoor,Both,0.60,1,438.0,40.1,1,Poorly Controlled
4,ASTH100004,21,Male,30.2,Never,0,None,Moderate,Active,Indoor,None,0.82,3,535.0,27.7,0,None


In [7]:
# Make columns snake_case (in memory)
df_all = pd.read_sql_query("SELECT * FROM asthma_records;", conn)
df_all.columns = [c.strip().lower().replace(" ", "_") for c in df_all.columns]

print("Rows:", len(df_all), "| Cols:", len(df_all.columns))
print("Columns:", list(df_all.columns))
display(df_all.describe(include='all').transpose().head(20))

Rows: 10000 | Cols: 17
Columns: ['patient_id', 'age', 'gender', 'bmi', 'smoking_status', 'family_history', 'allergies', 'air_pollution_level', 'physical_activity_level', 'occupation_type', 'comorbidities', 'medication_adherence', 'number_of_er_visits', 'peak_expiratory_flow', 'feno_level', 'has_asthma', 'asthma_control_level']


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
patient_id,10000,10000,ASTH100000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,10000.0,NaN,NaN,NaN,44.9307,25.653559,1.0,23.0,45.0,67.0,89.0
gender,10000,3,Female,4814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bmi,10000.0,NaN,NaN,NaN,25.05332,4.874466,15.0,21.6,25.0,28.4,45.0
smoking_status,10000,3,Never,6070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
family_history,10000.0,NaN,NaN,NaN,0.3034,0.459749,0.0,0.0,0.0,1.0,1.0
allergies,7064,4,Dust,2479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
air_pollution_level,10000,3,Moderate,4915,NaN,NaN,NaN,NaN,NaN,NaN,NaN
physical_activity_level,10000,3,Sedentary,4062,NaN,NaN,NaN,NaN,NaN,NaN,NaN
occupation_type,10000,2,Indoor,7035,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Adjust these names if your dataset differs; use PRAGMA schema if unsure
gender_col = "gender" if "gender" in df_all.columns else None
age_col    = "age"    if "age"    in df_all.columns else None

kpis = {}
if gender_col:
    kpis["gender_counts"] = df_all[gender_col].value_counts(dropna=False).to_dict()
if age_col:
    bins = [0,18,35,50,70,120]
    labels = ['0-18','19-35','36-50','51-70','71+']
    df_all["age_group"] = pd.cut(df_all[age_col], bins=bins, labels=labels, right=True, include_lowest=True)
    kpis["age_distribution"] = df_all["age_group"].value_counts().sort_index().to_dict()

pd.DataFrame([kpis])

,gender_counts,age_distribution
0,"{'Female': 4814, 'Male': 4786, 'Other': 400}","{'0-18': 2019, '19-35': 1940, '36-50': 1655, '..."


In [9]:
# Example clinical signals; tweak to match your columns (e.g., fev1, pef, eosinophils, hba1c, smoking_status)
out = {}

if "smoking_status" in df_all.columns:
    out["smokers"] = df_all["smoking_status"].value_counts().to_dict()

if "fev1" in df_all.columns:  # forced expiratory volume (if present)
    out["fev1_summary"] = df_all["fev1"].describe().to_dict()

if "hba1c" in df_all.columns:
    out["hba1c_outliers_over_6.5"] = int((df_all["hba1c"] > 6.5).sum())

pd.DataFrame([out])

,smokers
0,"{'Never': 6070, 'Former': 2487, 'Current': 1443}"


In [10]:
os.makedirs("reports/figures", exist_ok=True)

# Gender & age distribution CSVs (if available)
if "gender" in df_all.columns:
    df_all["gender"].value_counts(dropna=False).rename_axis("gender").reset_index(name="count") \
        .to_csv("reports/figures/gender_counts.csv", index=False)

if "age_group" in df_all.columns:
    df_all["age_group"].value_counts().sort_index().rename_axis("age_group").reset_index(name="count") \
        .to_csv("reports/figures/age_distribution.csv", index=False)

print("✅ Exported: reports/figures/gender_counts.csv (if gender present)")
print("✅ Exported: reports/figures/age_distribution.csv (if age present)")

✅ Exported: reports/figures/gender_counts.csv (if gender present)
✅ Exported: reports/figures/age_distribution.csv (if age present)


In [11]:
conn.close()
print("Closed SQLite connection.")

Closed SQLite connection.
